In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
import tensorflow as tf
from tensorflow.keras import layers, models
import tkinter as tk
from tkinter import ttk, messagebox

# Load the dataset
file_path = 'D:/Intelligent System/DATA/data_pttk_dl.csv'
data = pd.read_csv(file_path)

# Extract features and target variable
X = data[['10%', '20%', '20%_1']].values
y = data['Thi'].values

# Standardize the features
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

# Prepare data for CNN and RNN
X_train_cnn_rnn = X_train_scaled.reshape(-1, 3, 1)
X_test_cnn_rnn = X_test_scaled.reshape(-1, 3, 1)

### CNN without Keras ###
class SimpleCNN(tf.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = tf.Variable(tf.random.normal([2, 1, 32], dtype=tf.float32))
        self.dense1 = tf.Variable(tf.random.normal([96, 1], dtype=tf.float32))

    def __call__(self, x):
        conv_out = tf.nn.conv1d(x, self.conv1, stride=1, padding='SAME')
        conv_out = tf.nn.relu(conv_out)
        flat_out = tf.reshape(conv_out, [-1, 96])
        return tf.matmul(flat_out, self.dense1)

cnn_model = SimpleCNN()
optimizer_cnn = tf.optimizers.Adam()

for epoch in range(500):
    with tf.GradientTape() as tape:
        predictions_cnn = cnn_model(X_train_cnn_rnn)
        loss = tf.reduce_mean(tf.square(predictions_cnn - y_train.reshape(-1, 1).astype(np.float32)))
    grads = tape.gradient(loss, cnn_model.trainable_variables)
    optimizer_cnn.apply_gradients(zip(grads, cnn_model.trainable_variables))

y_pred_cnn = cnn_model(X_test_cnn_rnn).numpy().ravel()

### RNN without Keras ###
class SimpleRNN(tf.Module):
    def __init__(self):
        super(SimpleRNN, self).__init__()
        # Sử dụng ma trận trọng số Wx có kích thước phù hợp với số đặc trưng (3 đặc trưng)
        self.Wx = tf.Variable(tf.random.normal([1, 32]), name='Wx')  # 1 ở đây vì mỗi thời điểm là một vector với 1 đặc trưng (sau reshape)
        self.Wh = tf.Variable(tf.random.normal([32, 32]), name='Wh')
        self.b = tf.Variable(tf.zeros([32]), name='b')
        self.dense1 = tf.Variable(tf.random.normal([32, 1], dtype=tf.float32))

    def __call__(self, x):
        batch_size = tf.shape(x)[0]
        state = tf.zeros([batch_size, 32], dtype=tf.float32)  # Kích thước trạng thái RNN
        for t in range(x.shape[1]):  # Vòng lặp qua các thời điểm của chuỗi
            input_at_t = x[:, t, :]  # Lấy dữ liệu tại thời điểm t với kích thước [batch_size, 1]
            input_at_t = tf.reshape(input_at_t, [batch_size, 1])  # Đảm bảo reshape thành [batch_size, 1] cho phép nhân ma trận
            state = tf.tanh(tf.matmul(input_at_t, self.Wx) + tf.matmul(state, self.Wh) + self.b)
        return tf.matmul(state, self.dense1)

        
rnn_model = SimpleRNN()
optimizer_rnn = tf.optimizers.Adam()

for epoch in range(500):
    with tf.GradientTape() as tape:
        predictions_rnn = rnn_model(X_train_cnn_rnn)
        loss_rnn = tf.reduce_mean(tf.square(predictions_rnn - y_train.reshape(-1, 1).astype(np.float32)))
    grads_rnn = tape.gradient(loss_rnn, rnn_model.trainable_variables)
    optimizer_rnn.apply_gradients(zip(grads_rnn, rnn_model.trainable_variables))

y_pred_rnn = rnn_model(X_test_cnn_rnn).numpy().ravel()
### CNN with Keras ###
cnn_keras_model = models.Sequential([
    layers.Conv1D(32, 2, activation='relu', input_shape=(3, 1)),
    layers.Flatten(),
    layers.Dense(1)
])

cnn_keras_model.compile(optimizer='adam', loss='mean_squared_error')
cnn_keras_model.fit(X_train_cnn_rnn, y_train, epochs=500, verbose=0)
y_pred_cnn_keras = cnn_keras_model.predict(X_test_cnn_rnn).ravel()

### RNN with Keras ###
rnn_keras_model = models.Sequential([
    layers.SimpleRNN(32, input_shape=(3, 1)),
    layers.Dense(1)
])

rnn_keras_model.compile(optimizer='adam', loss='mean_squared_error')
rnn_keras_model.fit(X_train_cnn_rnn, y_train, epochs=500, verbose=0)
y_pred_rnn_keras = rnn_keras_model.predict(X_test_cnn_rnn).ravel()

### GUI Setup ###
def display_results():
    try:
        value1 = float(entry_10.get())
        value2 = float(entry_20.get())
        value3 = float(entry_20_1.get())
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numerical values.")
        return

    input_data = scaler.transform([[value1, value2, value3]])
    input_data_cnn_rnn = input_data.reshape(-1, 3, 1).astype(np.float32)

    selected_model = model_var.get()
    if selected_model == "CNN (non-Keras)":
        y_pred = cnn_model(input_data_cnn_rnn).numpy().ravel()
    elif selected_model == "RNN (non-Keras)":
        y_pred = rnn_model(input_data_cnn_rnn).numpy().ravel()
    elif selected_model == "CNN (Keras)":
        y_pred = cnn_keras_model.predict(input_data_cnn_rnn).ravel()
    elif selected_model == "RNN (Keras)":
        y_pred = rnn_keras_model.predict(input_data_cnn_rnn).ravel()
    else:
        messagebox.showerror("Model Selection Error", "Please select a model.")
        return

    messagebox.showinfo("Prediction Result", f"Predicted 'Thi': {y_pred[0]:.2f}")

def plot_comparison():
    models = ['CNN (non-Keras)', 'RNN (non-Keras)', 'CNN (Keras)', 'RNN (Keras)']
    
    mae_scores = [mean_absolute_error(y_test, y_pred_cnn),
                  mean_absolute_error(y_test, y_pred_rnn),
                  mean_absolute_error(y_test, y_pred_cnn_keras),
                  mean_absolute_error(y_test, y_pred_rnn_keras)]

    rmse_scores = [np.sqrt(mean_squared_error(y_test, y_pred_cnn)),
                   np.sqrt(mean_squared_error(y_test, y_pred_rnn)),
                   np.sqrt(mean_squared_error(y_test, y_pred_cnn_keras)),
                   np.sqrt(mean_squared_error(y_test, y_pred_rnn_keras))]

    plt.figure(figsize=(12, 6))

    # MAE Comparison
    plt.subplot(1, 2, 1)
    plt.bar(models, mae_scores, color=['blue', 'green', 'red', 'purple'])
    plt.title('MAE Comparison')
    plt.ylabel('MAE')

    # RMSE Comparison
    plt.subplot(1, 2, 2)
    plt.bar(models, rmse_scores, color=['blue', 'green', 'red', 'purple'])
    plt.title('RMSE Comparison')
    plt.ylabel('RMSE')

    plt.tight_layout()
    plt.show()

# Main window
root = tk.Tk()
root.title("Machine Learning Prediction")

# Input fields
tk.Label(root, text="10%:").grid(row=0, column=0)
entry_10 = tk.Entry(root)
entry_10.grid(row=0, column=1)

tk.Label(root, text="20%:").grid(row=1, column=0)
entry_20 = tk.Entry(root)
entry_20.grid(row=1, column=1)

tk.Label(root, text="20%_1:").grid(row=2, column=0)
entry_20_1 = tk.Entry(root)
entry_20_1.grid(row=2, column=1)

# Model selection
tk.Label(root, text="Select Model:").grid(row=3, column=0)
model_var = tk.StringVar(value="CNN (non-Keras)")
model_menu = ttk.Combobox(root, textvariable=model_var, values=["CNN (non-Keras)", "RNN (non-Keras)", "CNN (Keras)", "RNN (Keras)"])
model_menu.grid(row=3, column=1)

# Buttons
btn_predict = tk.Button(root, text="Predict", command=display_results)
btn_predict.grid(row=4, column=0, columnspan=2)

btn_plot = tk.Button(root, text="Show Comparison Chart", command=plot_comparison)
btn_plot.grid(row=5, column=0, columnspan=2)

# Run the GUI
root.mainloop()


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Intelligent System/DATA/data_pttk_dl.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
import tensorflow as tf
from tensorflow.keras import layers, models
import tkinter as tk
from tkinter import ttk, messagebox

# Load the dataset
file_path = 'D:/Intelligent System/DATA/data_pttk_dl.csv'
data = pd.read_csv(file_path)

# Extract features and target variable
X = data[['10%', '20%', '20%_1']].values
y = data['Thi'].values

# Standardize the features
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

# Prepare data for CNN and RNN
X_train_cnn_rnn = X_train_scaled.reshape(-1, 3, 1)
X_test_cnn_rnn = X_test_scaled.reshape(-1, 3, 1)

### CNN without Keras ###
class SimpleCNN(tf.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = tf.Variable(tf.random.normal([2, 1, 32], dtype=tf.float32))
        self.dense1 = tf.Variable(tf.random.normal([96, 1], dtype=tf.float32))

    def __call__(self, x):
        conv_out = tf.nn.conv1d(x, self.conv1, stride=1, padding='SAME')
        conv_out = tf.nn.relu(conv_out)
        flat_out = tf.reshape(conv_out, [-1, 96])
        return tf.matmul(flat_out, self.dense1)

cnn_model = SimpleCNN()
optimizer_cnn = tf.optimizers.Adam()

for epoch in range(500):
    with tf.GradientTape() as tape:
        predictions_cnn = cnn_model(X_train_cnn_rnn)
        loss = tf.reduce_mean(tf.square(predictions_cnn - y_train.reshape(-1, 1).astype(np.float32)))
    grads = tape.gradient(loss, cnn_model.trainable_variables)
    optimizer_cnn.apply_gradients(zip(grads, cnn_model.trainable_variables))

y_pred_cnn = cnn_model(X_test_cnn_rnn).numpy().ravel()

### RNN without Keras ###
class SimpleRNN(tf.Module):
    def __init__(self):
        super(SimpleRNN, self).__init__()
        # Sử dụng ma trận trọng số Wx có kích thước phù hợp với số đặc trưng (3 đặc trưng)
        self.Wx = tf.Variable(tf.random.normal([1, 32]), name='Wx')  # 1 ở đây vì mỗi thời điểm là một vector với 1 đặc trưng (sau reshape)
        self.Wh = tf.Variable(tf.random.normal([32, 32]), name='Wh')
        self.b = tf.Variable(tf.zeros([32]), name='b')
        self.dense1 = tf.Variable(tf.random.normal([32, 1], dtype=tf.float32))

    def __call__(self, x):
        batch_size = tf.shape(x)[0]
        state = tf.zeros([batch_size, 32], dtype=tf.float32)  # Kích thước trạng thái RNN
        for t in range(x.shape[1]):  # Vòng lặp qua các thời điểm của chuỗi
            input_at_t = x[:, t, :]  # Lấy dữ liệu tại thời điểm t với kích thước [batch_size, 1]
            input_at_t = tf.reshape(input_at_t, [batch_size, 1])  # Đảm bảo reshape thành [batch_size, 1] cho phép nhân ma trận
            state = tf.tanh(tf.matmul(input_at_t, self.Wx) + tf.matmul(state, self.Wh) + self.b)
        return tf.matmul(state, self.dense1)

        
rnn_model = SimpleRNN()
optimizer_rnn = tf.optimizers.Adam()

for epoch in range(500):
    with tf.GradientTape() as tape:
        predictions_rnn = rnn_model(X_train_cnn_rnn)
        loss_rnn = tf.reduce_mean(tf.square(predictions_rnn - y_train.reshape(-1, 1).astype(np.float32)))
    grads_rnn = tape.gradient(loss_rnn, rnn_model.trainable_variables)
    optimizer_rnn.apply_gradients(zip(grads_rnn, rnn_model.trainable_variables))

y_pred_rnn = rnn_model(X_test_cnn_rnn).numpy().ravel()
### CNN with Keras ###
cnn_keras_model = models.Sequential([
    layers.Conv1D(32, 2, activation='relu', input_shape=(3, 1)),
    layers.Flatten(),
    layers.Dense(1)
])

cnn_keras_model.compile(optimizer='adam', loss='mean_squared_error')
cnn_keras_model.fit(X_train_cnn_rnn, y_train, epochs=500, verbose=0)
y_pred_cnn_keras = cnn_keras_model.predict(X_test_cnn_rnn).ravel()

### RNN with Keras ###
rnn_keras_model = models.Sequential([
    layers.SimpleRNN(32, input_shape=(3, 1)),
    layers.Dense(1)
])

rnn_keras_model.compile(optimizer='adam', loss='mean_squared_error')
rnn_keras_model.fit(X_train_cnn_rnn, y_train, epochs=500, verbose=0)
y_pred_rnn_keras = rnn_keras_model.predict(X_test_cnn_rnn).ravel()

### GUI Setup ###
def display_results():
    try:
        value1 = float(entry_10.get())
        value2 = float(entry_20.get())
        value3 = float(entry_20_1.get())
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numerical values.")
        return

    input_data = scaler.transform([[value1, value2, value3]])
    input_data_cnn_rnn = input_data.reshape(-1, 3, 1).astype(np.float32)

    selected_model = model_var.get()
    if selected_model == "CNN (non-Keras)":
        y_pred = cnn_model(input_data_cnn_rnn).numpy().ravel()
    elif selected_model == "RNN (non-Keras)":
        y_pred = rnn_model(input_data_cnn_rnn).numpy().ravel()
    elif selected_model == "CNN (Keras)":
        y_pred = cnn_keras_model.predict(input_data_cnn_rnn).ravel()
    elif selected_model == "RNN (Keras)":
        y_pred = rnn_keras_model.predict(input_data_cnn_rnn).ravel()
    else:
        messagebox.showerror("Model Selection Error", "Please select a model.")
        return

    messagebox.showinfo("Prediction Result", f"Predicted 'Thi': {y_pred[0]:.2f}")

def plot_comparison():
    models = ['CNN (non-Keras)', 'RNN (non-Keras)', 'CNN (Keras)', 'RNN (Keras)']
    
    epsilon = 1e-10  # Small constant to avoid division by zero
    
    # Calculate MAE
    mae_scores = [mean_absolute_error(y_test, y_pred_cnn),
                  mean_absolute_error(y_test, y_pred_rnn),
                  mean_absolute_error(y_test, y_pred_cnn_keras),
                  mean_absolute_error(y_test, y_pred_rnn_keras)]
    
    # Calculate RMSE
    rmse_scores = [np.sqrt(mean_squared_error(y_test, y_pred_cnn)),
                   np.sqrt(mean_squared_error(y_test, y_pred_rnn)),
                   np.sqrt(mean_squared_error(y_test, y_pred_cnn_keras)),
                   np.sqrt(mean_squared_error(y_test, y_pred_rnn_keras))]
    
    # Calculate RME (Relative Mean Error)
    rme_scores = [np.mean(np.abs((y_test - y_pred_cnn) / (y_test + epsilon))),
                  np.mean(np.abs((y_test - y_pred_rnn) / (y_test + epsilon))),
                  np.mean(np.abs((y_test - y_pred_cnn_keras) / (y_test + epsilon))),
                  np.mean(np.abs((y_test - y_pred_rnn_keras) / (y_test + epsilon)))]
    
    # Calculate MAPE (Mean Absolute Percentage Error)
    mape_scores = [np.mean(np.abs((y_test - y_pred_cnn) / (y_test + epsilon))) * 100,
                   np.mean(np.abs((y_test - y_pred_rnn) / (y_test + epsilon))) * 100,
                   np.mean(np.abs((y_test - y_pred_cnn_keras) / (y_test + epsilon))) * 100,
                   np.mean(np.abs((y_test - y_pred_rnn_keras) / (y_test + epsilon))) * 100]

    # Plotting the comparison chart
    plt.figure(figsize=(14, 8))

    # MAE Comparison
    plt.subplot(2, 2, 1)
    plt.bar(models, mae_scores, color=['blue', 'green', 'red', 'purple'])
    plt.title('MAE Comparison')
    plt.ylabel('MAE')

    # RMSE Comparison
    plt.subplot(2, 2, 2)
    plt.bar(models, rmse_scores, color=['blue', 'green', 'red', 'purple'])
    plt.title('RMSE Comparison')
    plt.ylabel('RMSE')

    # RME Comparison
    plt.subplot(2, 2, 3)
    plt.bar(models, rme_scores, color=['blue', 'green', 'red', 'purple'])
    plt.title('RME Comparison')
    plt.ylabel('RME')

    # MAPE Comparison
    plt.subplot(2, 2, 4)
    plt.bar(models, mape_scores, color=['blue', 'green', 'red', 'purple'])
    plt.title('MAPE Comparison')
    plt.ylabel('MAPE (%)')

    plt.tight_layout()
    plt.show()

# Main window
root = tk.Tk()
root.title("Machine Learning Prediction")

# Input fields
tk.Label(root, text="10%:").grid(row=0, column=0)
entry_10 = tk.Entry(root)
entry_10.grid(row=0, column=1)

tk.Label(root, text="20%:").grid(row=1, column=0)
entry_20 = tk.Entry(root)
entry_20.grid(row=1, column=1)

tk.Label(root, text="20%_1:").grid(row=2, column=0)
entry_20_1 = tk.Entry(root)
entry_20_1.grid(row=2, column=1)

# Model selection
tk.Label(root, text="Select Model:").grid(row=3, column=0)
model_var = tk.StringVar(value="CNN (non-Keras)")
model_menu = ttk.Combobox(root, textvariable=model_var, values=["CNN (non-Keras)", "RNN (non-Keras)", "CNN (Keras)", "RNN (Keras)"])
model_menu.grid(row=3, column=1)

# Buttons
btn_predict = tk.Button(root, text="Predict", command=display_results)
btn_predict.grid(row=4, column=0, columnspan=2)

btn_plot = tk.Button(root, text="Show Comparison Chart", command=plot_comparison)
btn_plot.grid(row=5, column=0, columnspan=2)

# Run the GUI
root.mainloop()
